# Compute scores

In [1]:
import pandas as pd

from config import QUANTILES, ROOT
from src.load_data import load_nowcasts, load_predictions
from src.scoring_functions import evaluate_models

In [2]:
df = load_predictions()

In [3]:
df_nowcasts = load_nowcasts(quantiles=QUANTILES)
df = pd.concat([df, df_nowcasts], ignore_index=True)

In [4]:
df.model.unique()

array(['Ensemble', 'hhh4', 'hhh4-Discard', 'hhh4-Naive', 'hhh4-NoCovid',
       'hhh4-Oracle', 'hhh4-Shuffle', 'hhh4-Vincentization', 'Historical',
       'LightGBM', 'LightGBM-Discard', 'LightGBM-Naive',
       'LightGBM-NoCovariates', 'LightGBM-NoCovid', 'LightGBM-Oracle',
       'Persistence', 'TSCount-Extended', 'TSCount-Simple',
       'TSMixer-Covariates', 'TSMixer', 'TSMixer-Discard',
       'TSMixer-Naive', 'TSMixer-Oracle', 'TSMixer-NoCovid', 'Nowcast'],
      dtype=object)

# Aggregated scores

In [5]:
df_national = evaluate_models(df, "national")
df_age = evaluate_models(df, "age")

scores = pd.concat(
    [df_national.assign(level="national"), df_age.assign(level="age")],
    ignore_index=True,
)

scores["level"] = pd.Categorical(
    scores["level"], categories=["national", "age"], ordered=True
)

In [6]:
scores.to_csv(ROOT / 'results' / 'scores' / 'scores.csv', float_format="%.3f", index=False)

## By age group

In [7]:
scores_age = evaluate_models(df, "age", by_age=True)

In [8]:
scores_age.to_csv(ROOT / 'results' / 'scores' / 'scores_age.csv', float_format="%.3f", index=False)

## By horizon

In [9]:
df_national = evaluate_models(df, "national", by_horizon=True)
df_age = evaluate_models(df, "age", by_horizon=True)

scores_horizon = pd.concat(
    [df_national.assign(level="national"), df_age.assign(level="age")],
    ignore_index=True,
)

In [10]:
scores_horizon.to_csv(ROOT / 'results' / 'scores' / 'scores_horizon.csv', float_format="%.3f", index=False)